In [0]:
##Setup

In [0]:
import tensorflow as tf

import numpy as np
import os
import time

In [0]:
#downloading the data


In [3]:
path_to_file = tf.keras.utils.get_file('male.txt', 'http://www.cs.cmu.edu/afs/cs/project/ai-repository/ai/areas/nlp/corpora/names/male.txt')

24576/20466 [====================================] - 0s 15us/step


In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 20466 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

# List of common male names.
# Copyright (c) January 1991 by Mark Kantrowitz.
# 2940 names
# Thanks to Bill Ross for about 1000 additional names.
# Version 1.3 (29-MAR-94)

Aamir
Aaron
Abbey
Abbie
Abbot
Abbott
Abby
Abdel
Abdul
Abdulkarim
Abdullah
Abe


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))
print(vocab[:64])

65 unique characters
['\n', ' ', '#', '(', ')', '-', '.', '0', '1', '2', '3', '4', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y']


In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
print(char2idx)
idx2char = np.array(vocab)
print(idx2char)

text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int)

{'\n': 0, ' ': 1, '#': 2, '(': 3, ')': 4, '-': 5, '.': 6, '0': 7, '1': 8, '2': 9, '3': 10, '4': 11, '9': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
['\n' ' ' '#' '(' ')' '-' '.' '0' '1' '2' '3' '4' '9' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']
[ 2  1 24 ... 46 59  0]


In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '#' :   2,
  '(' :   3,
  ')' :   4,
  '-' :   5,
  '.' :   6,
  '0' :   7,
  '1' :   8,
  '2' :   9,
  '3' :  10,
  '4' :  11,
  '9' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [9]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'# List of com' ---- characters mapped to int ---- > [ 2  1 24 47 57 58  1 53 44  1 41 53 51]


In [10]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

#
 
L
i
s


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'# List of common male names.\n# Copyright (c) January 1991 by Mark Kantrowitz.\n# 2940 names\n# Thanks t'
'o Bill Ross for about 1000 additional names.\n# Version 1.3 (29-MAR-94)\n\nAamir\nAaron\nAbbey\nAbbie\nAbbot'
'\nAbbott\nAbby\nAbdel\nAbdul\nAbdulkarim\nAbdullah\nAbe\nAbel\nAbelard\nAbner\nAbraham\nAbram\nAce\nAdair\nAdam\nAdam'
's\nAddie\nAdger\nAditya\nAdlai\nAdnan\nAdolf\nAdolfo\nAdolph\nAdolphe\nAdolpho\nAdolphus\nAdrian\nAdrick\nAdrien\nAg'
'amemnon\nAguinaldo\nAguste\nAgustin\nAharon\nAhmad\nAhmed\nAhmet\nAjai\nAjay\nAl\nAlaa\nAlain\nAlan\nAlasdair\nAlast'


In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '# List of common male names.\n# Copyright (c) January 1991 by Mark Kantrowitz.\n# 2940 names\n# Thanks '
Target data: ' List of common male names.\n# Copyright (c) January 1991 by Mark Kantrowitz.\n# 2940 names\n# Thanks t'


In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 2 ('#')
  expected output: 1 (' ')
Step    1
  input: 1 (' ')
  expected output: 24 ('L')
Step    2
  input: 24 ('L')
  expected output: 47 ('i')
Step    3
  input: 47 ('i')
  expected output: 57 ('s')
Step    4
  input: 57 ('s')
  expected output: 58 ('t')


In [15]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [22]:
sampled_indices

array([28, 53,  0, 22, 49, 25, 33, 38,  0, 12, 32, 35, 42, 37, 41, 18, 49,
       37,  0, 13, 57,  0, 31,  3, 23, 13, 64, 64, 30, 60, 63, 61, 50,  1,
       46, 31, 14, 16, 51, 35, 39, 55, 22, 22, 29, 42, 40,  6, 38, 58,  0,
       51, 11, 26, 30, 60, 56, 27, 44, 44, 32, 50, 30, 28, 40, 38, 14, 20,
       30, 38, 36, 64, 10, 43, 60, 18,  6,  5, 25, 32, 14,  2, 43, 32, 49,
       60, 25,  9, 26, 38,  3, 30,  1, 63, 13, 32, 54, 19,  2, 50])

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'tus\nCleveland\nCliff\nClifford\nClifton\nClint\nClinten\nClinton\nClive\nClyde\nCob\nCobb\nCobbie\nCobby\nCody\nCo'

Next Char Predictions: 
 'Po\nJkMUZ\n9TWdYcFkY\nAs\nS(KAzzRvywl hSBDmWaqJJQdb.Zt\nm4NRvrOffTlRPbZBHRZXz3evF.-MTB#eTkvM2NZ(R yATpG#l'


In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.174549


In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
EPOCHS=10


In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
3/3 [==============================] - 13s 4s/step - loss: 3.9828
Epoch 2/10
3/3 [==============================] - 14s 5s/step - loss: 3.5968
Epoch 3/10
3/3 [==============================] - 14s 5s/step - loss: 3.3681
Epoch 4/10
3/3 [==============================] - 14s 5s/step - loss: 3.2923
Epoch 5/10
3/3 [==============================] - 14s 5s/step - loss: 3.2710
Epoch 6/10
3/3 [==============================] - 14s 5s/step - loss: 3.2532
Epoch 7/10
3/3 [==============================] - 14s 5s/step - loss: 3.2408
Epoch 8/10
3/3 [==============================] - 14s 5s/step - loss: 3.2190
Epoch 9/10
3/3 [==============================] - 14s 5s/step - loss: 3.2049
Epoch 10/10
3/3 [==============================] - 13s 4s/step - loss: 3.1862


In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [34]:
print(generate_text(model, start_string="Bar"))

BarlTkowd

tnzt
ur
pmeSkPdciByboant
epnilNna
isoarilrbeoy
areRgeHRoi
cEwl
dee
oer
a
eRfl
ruTPeBrdiCiiRnaWhhRrolonava
ls
rucnreiedrPi

euntmcnin
azeooraLnrvasCW
c
gnyr
BSieTt
Oeat
S
Sytr
w
eoyuoadotnuiiLZGsgae
il

ia
teDnC
asoooeaniefN
ydolKfd
zt
Eyf-mradioasuuaeulBlioa
iiSilioSay
ocNshlleiSeF
JsleyLV

td
itLaSL
hi
Q
a
apy
JiGTon
r
l
foay
EeWtiJCinsi
Ueclnztk
e
oPyls
BalrcSynOHa
iDerr
HaanpTyadilagcenrrCNroeri
l
rErXrtoaudNhDlWrFS

r
elt
so
hoea
Del
A
I
iaS
aW
o
saeae
nHait
mie
KaiyeehaRoSt
illnEreoa
e
e
AdeatCrLncing

r
tGio
DMooCCtigllDe
ar
fi
n
LhroisoSaem
wrah
nrdottws
ArSnfrRod
alr
CeoanoapaiilusoSnieeiivnlssWTa
nesd
Pdsate
tDaerarDaCleLac
ilrtyo
wsa
eclnho
nRaO
Ilrt


ltlev
slvWrilsooredOli
nluar
blJKhieark
erSeDdrt 
RAlrt
eWl
SynthRI
csano
tsm
Jli
RenyyiSBrilnllarliRim
eee
srnls
r
rerA

CtTe
Jody
WtDl
repneeihsriicb
eatFmnoe
ay
vnsieomm
eitDCW
Wrrurrhn
h

cshohle
Jhoratal
tyaotntr
itld
nocprihR
llalrinter
rSlns
soraehooBte
podaeenr
BonObatiNuMJr

dn
m
riiRv
MPblaTeynBsiS
iWhsiTlD